In [ ]:
import numpy as np
import pandas as pd
import genetic_algorithm.population as population

pop = population.generate_random_forest(50, 5)

In [ ]:
import genetic_algorithm.transforms as transforms
data = pd.read_csv("../data/ES15.csv")
x_raw = data.values

transforms.get_oplist(pop[3])

In [ ]:
x_gen = transforms.forest2features(
    population=pop,
    x_raw=x_raw
)

In [ ]:
import genetic_algorithm.visualization as visualization

# assume x_gen is your data array of shape (n_samples, n_features)
x = pd.DataFrame(x_gen)


visualization.visualize_all_distributions(x=x)

In [ ]:
import genetic_algorithm.evaluation as evaluation

scoresdf, bestlist, scorelist = \
    evaluation.evaluate_forest(x_gen, x_raw[:, 3], n_bins=300,lag_range=(2,4))


print(scoresdf)
print(bestlist)

In [ ]:
oplist = transforms.get_oplist(pop[bestlist[0]])
print(oplist)
img = visualization.visualize_tree(pop[bestlist[0]])

In [ ]:
img

In [ ]:
import genetic_algorithm.population as population
import genetic_algorithm.mutation as mutation
import genetic_algorithm.transforms as transforms
import genetic_algorithm.optimize as optimize
import pandas as pd
import numpy as np
data = pd.read_csv("../data/ES15.csv")
x_raw = data.values

pop = population.generate_random_forest(50, 3)

np.seterr(all='ignore')
best_forest, best_scores, best_overtime = optimize.optimize_constants(
    pop, x_raw, sthresh_q=.25
)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(range(len(best_scores)), best_scores)
plt.show()

In [ ]:
best_scores[3]

In [ ]:
import numpy as np
x_ = transforms.forest2features(
    population=best_forest,
    x_raw=x_raw
)
y_ = np.pad(
    np.log(x_raw[1:, 3] / x_raw[:-1, 3]),
    (0,1),
    mode='constant',
    constant_values=0.0)

In [ ]:
print(np.std(y_))

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size=0.3)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from keras.optimizers.schedules import ExponentialDecay

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.95, 
    patience=20, 
    min_lr=1e-6
)

opt  = tf.keras.optimizers.Adam(learning_rate=0.02)
opt2 = tf.keras.optimizers.SGD(learning_rate=0.02)

def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='linear'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(64, activation='linear'),  
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='linear'),       
        tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
    ])
    
    rmse='root_mean_squared_error'

    model.compile(optimizer=opt, loss='mse', metrics=['mae','R2Score'])
    return model


epochs = 2000

with tf.device('/GPU:0'):
	model = build_model()
	history = model.fit(X_train, y_train, epochs=epochs, batch_size=512, \
                     validation_split=0.2, verbose=1, shuffle=True, callbacks=[reduce_lr])
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

plt.scatter(y_pred, y_test, s=1)
plt.grid()
plt.axis('tight')
plt.title('Testing Outputs')
plt.xlabel('y_pred')
plt.xlim(-0.015,0.015)
plt.ylim(-0.015,0.015)
plt.ylabel('y_test')
ax = plt.gca()
x_vals = np.array(ax.get_xlim())
y_vals = x_vals  # Since y = x
plt.plot(x_vals, y_vals, '-', color='black', label='y = x', linewidth=0.5)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0,color='black',linewidth=0.5)
plt.show()

In [ ]:
tp, fp, tn, fn = 0, 0, 0, 0
tp5, fp5, tn5, fn5 = 0, 0, 0, 0
for i in range(len(y_pred)):
    if(y_pred[i]>0):
        if(y_test[i]>0):
            tp+=1
        if(y_test[i]<0):
            fp+=1
        if(y_pred[i]>=5):
            if(y_test[i]>0):
                tp5+=1
            if(y_test[i]<0):
                fp5+=1
    if(y_pred[i]<0):
        if(y_test[i]<0):
            tn+=1
        if(y_test[i]>0):
            fn+=1
        if(y_pred[i]<=-5):
            if(y_test[i]<0):
                tn5+=1
            if(y_test[i]>0):
                fn5+=1
directionalAccuracy = ((tp+tn)/(tp+fp+tn+fn))*10000//1/100
print('Directional Accuracy:\t\t',directionalAccuracy)